In [4]:
pip install selenium beautifulsoup4 pandas openpyxl webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# === Step 1: Read Excel file ===
input_file = "Input.xlsx"
df = pd.read_excel(input_file)

# === Step 2: Setup Selenium WebDriver ===
chrome_options = Options()
chrome_options.add_argument("--headless")   # run in background (no browser window)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# === Step 3: Create output folder ===
output_folder = "articles"
os.makedirs(output_folder, exist_ok=True)

# === Step 4: Function to scrape article ===
def extract_article_with_selenium(url):
    try:
        driver.get(url)
        time.sleep(2)  # wait for page to load fully

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract article title (try <h1> first, fallback to <title>)
        title_tag = soup.find("h1") or soup.find("title")
        title = title_tag.get_text(strip=True) if title_tag else "No Title Found"

        # Extract article content (all <p> tags)
        paragraphs = soup.find_all("p")
        article_text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

        return title, article_text
    except Exception as e:
        print(f"❌ Error extracting {url}: {e}")
        return None, None

# === Step 5: Loop through URLs and save to .txt files ===
for _, row in df.iterrows():
    url_id = str(row["URL_ID"])
    url = row["URL"]

    print(f"Scraping: {url}")
    title, content = extract_article_with_selenium(url)

    if title and content:
        file_path = os.path.join(output_folder, f"{url_id}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(title + "\n\n" + content)
        print(f"✅ Saved: {file_path}")
    else:
        print(f"⚠️ Skipped: {url} (no content extracted)")

# === Step 6: Close driver ===
driver.quit()


Scraping: https://insights.blackcoffer.com/ai-and-ml-based-youtube-analytics-and-content-creation-tool-for-optimizing-subscriber-engagement-and-content-strategy/
✅ Saved: articles\Netclan20241017.txt
Scraping: https://insights.blackcoffer.com/enhancing-front-end-features-and-functionality-for-improved-user-experience-and-dashboard-accuracy-in-partner-hospital-application/
✅ Saved: articles\Netclan20241018.txt
Scraping: https://insights.blackcoffer.com/roas-dashboard-for-campaign-wise-google-ads-budget-tracking-using-google-ads-ap/
✅ Saved: articles\Netclan20241019.txt
Scraping: https://insights.blackcoffer.com/efficient-processing-and-analysis-of-financial-data-from-pdf-files-addressing-formatting-inconsistencies-and-ensuring-data-integrity-for-a-toyota-dealership-management-firm/
✅ Saved: articles\Netclan20241020.txt
Scraping: https://insights.blackcoffer.com/development-of-ea-robot-for-automated-trading/
✅ Saved: articles\Netclan20241021.txt
Scraping: https://insights.blackcoffer.com

In [14]:
pip install pandas numpy xlsxwriter openpyxl


Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import re
import pandas as pd
import numpy as np

# ======================
# 1️⃣ READ INPUT EXCEL
# ======================
input_excel = "Input.xlsx"   # Make sure Input.xlsx is in same folder
df_input = pd.read_excel(input_excel)

# ======================
# 2️⃣ SENTIMENT WORD LISTS (can be extended)
# ======================
# Replace with larger lexicon lists if available
positive_words = ["good", "great", "excellent", "positive", "fortunate", "correct", "superior", "happy", "benefit", "success"]
negative_words = ["bad", "terrible", "poor", "negative", "unfortunate", "wrong", "inferior", "sad", "problem", "failure"]

# ======================
# 3️⃣ HELPER FUNCTIONS
# ======================

def count_syllables(word):
    """Estimate syllables in a word (basic heuristic)."""
    word = word.lower()
    vowels = "aeiou"
    count = 0
    prev_char_was_vowel = False
    for char in word:
        if char in vowels:
            if not prev_char_was_vowel:
                count += 1
            prev_char_was_vowel = True
        else:
            prev_char_was_vowel = False
    if word.endswith("e"):
        count = max(1, count-1)
    return count if count > 0 else 1

def is_complex_word(word):
    """A complex word has 3 or more syllables."""
    return count_syllables(word) >= 3

def analyze_text(text):
    """Perform text analysis and return metrics as a dictionary."""

    # Sentence and word tokenization
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]

    words = re.findall(r'\b[a-zA-Z]+\b', text)
    words_lower = [w.lower() for w in words]

    word_count = len(words)
    sentence_count = len(sentences) if sentences else 1  # prevent divide by zero

    # Sentiment scores
    pos_score = sum(1 for w in words_lower if w in positive_words)
    neg_score = sum(1 for w in words_lower if w in negative_words)
    polarity = (pos_score - neg_score) / ((pos_score + neg_score) + 1e-6)
    subjectivity = (pos_score + neg_score) / (word_count + 1e-6)

    # Complexity metrics
    complex_words = [w for w in words if is_complex_word(w)]
    complex_word_count = len(complex_words)
    avg_sentence_length = word_count / sentence_count
    pct_complex_words = complex_word_count / word_count if word_count else 0
    fog_index = 0.4 * (avg_sentence_length + pct_complex_words * 100)

    # Other metrics
    syllable_per_word = np.mean([count_syllables(w) for w in words]) if words else 0
    personal_pronouns = len(re.findall(r"\b(I|we|my|ours|us)\b", text, flags=re.I))
    avg_word_length = np.mean([len(w) for w in words]) if words else 0

    return {
        "POSITIVE SCORE": pos_score,
        "NEGATIVE SCORE": neg_score,
        "POLARITY SCORE": polarity,
        "SUBJECTIVITY SCORE": subjectivity,
        "AVG SENTENCE LENGTH": avg_sentence_length,
        "PERCENTAGE OF COMPLEX WORDS": pct_complex_words,
        "FOG INDEX": fog_index,
        "AVG NUMBER OF WORDS PER SENTENCE": avg_sentence_length,
        "COMPLEX WORD COUNT": complex_word_count,
        "WORD COUNT": word_count,
        "SYLLABLE PER WORD": syllable_per_word,
        "PERSONAL PRONOUNS": personal_pronouns,
        "AVG WORD LENGTH": avg_word_length
    }

# ======================
# 4️⃣ PROCESS ARTICLES
# ======================
output_data = []

for _, row in df_input.iterrows():
    url_id = row["URL_ID"]
    url = row["URL"]
    file_path = os.path.join("articles", f"{url_id}.txt")

    if not os.path.exists(file_path):
        print(f"⚠️ Missing file for {url_id}")
        continue

    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    metrics = analyze_text(text)
    metrics["URL_ID"] = url_id
    metrics["URL"] = url
    output_data.append(metrics)

# ======================
# 5️⃣ CREATE FINAL DATAFRAME IN CORRECT ORDER
# ======================
output_df = pd.DataFrame(output_data)

final_columns = [
    "URL_ID", "URL",
    "POSITIVE SCORE", "NEGATIVE SCORE", "POLARITY SCORE", "SUBJECTIVITY SCORE",
    "AVG SENTENCE LENGTH", "PERCENTAGE OF COMPLEX WORDS", "FOG INDEX",
    "AVG NUMBER OF WORDS PER SENTENCE", "COMPLEX WORD COUNT", "WORD COUNT",
    "SYLLABLE PER WORD", "PERSONAL PRONOUNS", "AVG WORD LENGTH"
]

output_df = output_df[final_columns]

# ======================
# 6️⃣ SAVE TO UTF‑8 EXCEL FILE
# ======================
output_file = "Final_Output.xlsx"

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    output_df.to_excel(writer, index=False, sheet_name='Sheet1')
    # prevent Excel auto-converting strings to hyperlinks
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    for col_num, value in enumerate(output_df.columns.values):
        worksheet.write(0, col_num, value)


In [26]:
import os
import re
import pandas as pd
import numpy as np

# ===========================
# 1️⃣ HELPER FUNCTIONS
# ===========================

def clean_text(text, stopwords=None):
    """Remove stopwords and tokenize text into words"""
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    if stopwords:
        words = [w for w in words if w.lower() not in stopwords]
    return words

def count_syllables(word):
    """Count syllables in a word based on vowels with adjustments for 'es'/'ed' endings"""
    word = word.lower()
    vowels = "aeiou"
    count = 0
    prev_char_was_vowel = False
    for char in word:
        if char in vowels:
            if not prev_char_was_vowel:
                count += 1
            prev_char_was_vowel = True
        else:
            prev_char_was_vowel = False
    # Adjust for endings
    if word.endswith(("es","ed","e")):
        count = max(1, count-1)
    return count if count > 0 else 1

def is_complex_word(word):
    """A complex word has 3 or more syllables"""
    return count_syllables(word) >= 3

def analyze_text(text, positive_words, negative_words, stopwords=None):
    """Compute all metrics from Text Analysis.docx"""

    # ========= CLEAN TEXT =========
    words = clean_text(text, stopwords)
    word_count = len(words)

    # ========= SENTENCES =========
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    sentence_count = len(sentences) if sentences else 1

    # ========= SENTIMENT SCORES =========
    pos_score = sum(1 for w in words if w.lower() in positive_words)
    neg_score = sum(1 for w in words if w.lower() in negative_words)

    polarity = (pos_score - neg_score) / ((pos_score + neg_score) + 0.000001)
    subjectivity = (pos_score + neg_score) / (word_count + 0.000001)

    # ========= READABILITY =========
    avg_sentence_length = word_count / sentence_count
    complex_word_count = sum(1 for w in words if is_complex_word(w))
    pct_complex_words = complex_word_count / word_count if word_count else 0
    fog_index = 0.4 * (avg_sentence_length + (pct_complex_words * 100))

    # ========= OTHER METRICS =========
    syllable_per_word = np.mean([count_syllables(w) for w in words]) if words else 0
    personal_pronouns = len(re.findall(r"\b(I|we|my|ours|us)\b", text, flags=re.I))
    avg_word_length = np.mean([len(w) for w in words]) if words else 0

    return {
        "POSITIVE SCORE": pos_score,
        "NEGATIVE SCORE": neg_score,
        "POLARITY SCORE": polarity,
        "SUBJECTIVITY SCORE": subjectivity,
        "AVG SENTENCE LENGTH": avg_sentence_length,
        "PERCENTAGE OF COMPLEX WORDS": pct_complex_words,
        "FOG INDEX": fog_index,
        "AVG NUMBER OF WORDS PER SENTENCE": avg_sentence_length,
        "COMPLEX WORD COUNT": complex_word_count,
        "WORD COUNT": word_count,
        "SYLLABLE PER WORD": syllable_per_word,
        "PERSONAL PRONOUNS": personal_pronouns,
        "AVG WORD LENGTH": avg_word_length
    }

# ===========================
# 2️⃣ LOAD INPUT & STOPWORDS
# ===========================

input_excel = "Input.xlsx"
df_input = pd.read_excel(input_excel)

# Load stopwords from provided files
stopword_files = ["StopWords_Currencies.txt", "StopWords_Auditor.txt", "StopWords_DatesandNumbers.txt"]
stopwords = set()

for file in stopword_files:
    if os.path.exists(file):
        with open(file, "r", encoding="utf-8") as f:
            for line in f:
                word = line.strip().split('|')[0].strip()
                if word:
                    stopwords.add(word.lower())

# ===========================
# 3️⃣ SENTIMENT WORD LISTS
# (Replace with MasterDictionary if available)
# ===========================
positive_words = {"good","great","excellent","positive","fortunate","correct","superior","happy","benefit","success"}
negative_words = {"bad","terrible","poor","negative","unfortunate","wrong","inferior","sad","problem","failure"}

# ===========================
# 4️⃣ ANALYZE EACH ARTICLE
# ===========================
output_data = []

for _, row in df_input.iterrows():
    url_id = row["URL_ID"]
    url = row["URL"]
    file_path = os.path.join("articles", f"{url_id}.txt")

    if not os.path.exists(file_path):
        print(f"⚠️ Missing file for {url_id}")
        continue

    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    metrics = analyze_text(text, positive_words, negative_words, stopwords)
    metrics["URL_ID"] = url_id
    metrics["URL"] = url
    output_data.append(metrics)

# ===========================
# 5️⃣ CREATE DATAFRAME
# ===========================
output_df = pd.DataFrame(output_data)

final_columns = [
    "URL_ID", "URL",
    "POSITIVE SCORE", "NEGATIVE SCORE", "POLARITY SCORE", "SUBJECTIVITY SCORE",
    "AVG SENTENCE LENGTH", "PERCENTAGE OF COMPLEX WORDS", "FOG INDEX",
    "AVG NUMBER OF WORDS PER SENTENCE", "COMPLEX WORD COUNT", "WORD COUNT",
    "SYLLABLE PER WORD", "PERSONAL PRONOUNS", "AVG WORD LENGTH"
]

output_df = output_df[final_columns]

# ===========================
# 6️⃣ SAVE TO EXCEL + UTF‑8 CSV
# ===========================
excel_file = "Final_Output.xlsx"
csv_file = "Final_Output_UTF8.csv"

output_df.to_excel(excel_file, index=False, engine='xlsxwriter')
output_df.to_csv(csv_file, index=False, encoding='utf-8-sig')

print("✅ Final_Output.xlsx & Final_Output_UTF8.csv created successfully!")


✅ Final_Output.xlsx & Final_Output_UTF8.csv created successfully!
